# Homework 2
### Roman Blahuta
### ПКН18Б, 4th year

**NOTE**: .csv файл який використовується це конвертована Excel таблиця надана викладачем.

In [67]:
import altair as alt
import pandas as pd
import datetime

alt.data_transformers.enable('csv')

DataTransformerRegistry.enable('csv')

In [68]:
def extractDate(dateTimeString: str) -> str:
    return dateTimeString.split('-')[1].replace('.', '/')

def extractYear(dateTimeString: str) -> str:
    return dateTimeString.split('-')[1][-4:]

def extractHour(dateTimeString: str) -> str:
    return dateTimeString.split('-')[0]

def extractMonth(dateTimeString: str) -> str:
    return dateTimeString.split('-')[1].split('.')[1]

def extractWeekDay(dateString: str) -> str:
    return datetime.datetime.strptime(dateString, '%d/%m/%Y').strftime('%A')

In [69]:
df = pd.read_csv('../data/2014-2020.csv')
df = df.drop('Unnamed: 12', 1)
df['Date'] = df['Час/Дата'].apply(extractDate)
df['Hour'] = df['Час/Дата'].apply(extractHour)
df['Year'] = df['Час/Дата'].apply(extractYear)
df['Month'] = df["Час/Дата"].apply(extractMonth)
df['Weekday'] = df['Date'].apply(extractWeekDay)
df.head(8000)

/tmp/ipykernel_12253/3818695156.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop('Unnamed: 12', 1)


,Час/Дата,AES,TEC,VDE,TES,GES,GAES_GEN,CONSUMPTION,GAES_PUMP,UK_BLR_RUS,UK_EURO,UK_MLD,Date,Hour,Year,Month,Weekday
0,24-31.12.2020,9235,2039,621,4942,385.0,0,16693,-405.0,-84.0,-11.0,-29.0,31/12/2020,24,2020,12,Thursday
1,23-31.12.2020,9221,2159,707,5549,470.0,0,17805,0.0,-43.0,-212.0,-46.0,31/12/2020,23,2020,12,Thursday
2,22-31.12.2020,9249,2377,709,5906,1000.0,0,18870,0.0,-13.0,-328.0,-30.0,31/12/2020,22,2020,12,Thursday
3,21-31.12.2020,9256,2499,702,6329,909.0,322,19887,0.0,0.0,-64.0,-66.0,31/12/2020,21,2020,12,Thursday
4,20-31.12.2020,9213,2521,702,6640,823.0,602,20387,0.0,-29.0,-48.0,-37.0,31/12/2020,20,2020,12,Thursday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,21-02.02.2020,10542,2523,708,2950,1675.0,0,18550,0.0,418.0,-142.0,-124.0,02/02/2020,21,2020,02,Sunday
7996,20-02.02.2020,10531,2541,721,3033,1491.0,280,18812,0.0,336.0,-75.0,-46.0,02/02/2020,20,2020,02,Sunday
7997,19-02.02.2020,10536,2537,698,2967,1473.0,619,19150,0.0,339.0,43.0,-62.0,02/02/2020,19,2020,02,Sunday
7998,18-02.02.2020,10546,2535,689,2993,1934.0,332,19219,0.0,374.0,-7.0,-177.0,02/02/2020,18,2020,02,Sunday


### 1. Як змінювалась структура генерації електроенергії за роками?

Я спинився на варіанті звичайного лінійного графіка оскільки вважаю його максимально інтуітивним та найбільш знайомим видом візуалізації для пересічного користувача. Крім того нахил ліній додатково підкреслює злети та спади у генерації електроенергії. Для відображення взаємозвязку даних "Кількість Електроенергії"/"Рік" більш ніж достатньо простого лінійного графіка. Для легшого користування Tooltip я додатково виділив точки із значеннями на кожній лінії.

Серед альтернативних варіантів я розглядав стовпцеві діаграми, а саме Stacked Bar Chart та Layered Bar Chart. 

Перший із цих варіантів мені здався менш інтуітивно зрозумілим та потребував постійного обрахунку відносно шкали (віднімати початкове значення стовпця від кінцевого).

Layered Bar Chart насправді був варіантом який я розглядав спочатку, але через обмеження інструменту Altair я мусив відкинути цей варіант. Проблемою виявилась відсутність (принаймні наскільки мені відомо) можливості сортування порядку шарів діаграми відповідно до значень стовпців. Приклад: шар із значеннями даних VDE містить найнижчі показники та стовпці і інші шари діаграми просто-напросто перекривають шар VDE. Варіант із вказанням прозорості шарів видався мені не надто візуально приємним та може плутати користувачів, оскільки кольори дещо зливаються.

Дані були згруповані та просумовані згідно стовпця Year.

In [49]:
dfByYear = df.groupby(['Year']).sum().reset_index()
dfByYear

,Year,AES,TEC,VDE,TES,GES,GAES_GEN,CONSUMPTION,GAES_PUMP,UK_BLR_RUS,UK_EURO,UK_MLD
0,2014,88204418,14684731,1606386,68605877,7991074.0,839842,172595823,-1137978.0,-2597117.0,-4945382.0,-654735.0
1,2015,87413763,12041074,1234936,50259819,4964478.0,1554857,148380125,-2116247.0,1985549.0,-3623023.0,52187.0
2,2016,80762094,12841033,1093190,49879543,7113989.0,1621377,145108274,-2180363.0,-299496.0,-3983215.0,45211.0
3,2017,85314258,12208062,1177032,44945303,8531952.0,1575099,145483538,-2138632.0,-315514.0,-4038848.0,-1153638.0
4,2018,84351328,12261018,1863726,47747092,10072767.0,1564867,149131882,-2119310.0,-433159.0,-5210685.0,-965754.0
5,2019,83098265,12251850,4441801,44877587,6422154.0,1323192,146253834,-1819507.0,562127.0,-4280178.0,-623457.0
6,2020,76210883,14536545,8124734,39504024,5998680.0,1561564,140720453,-2139482.0,-638700.0,-2300524.0,-137293.0


In [105]:
chart1Line = alt.Chart(dfByYear, title='Як змінювалась структура генерації електроенергії за роками?').transform_fold(
    fold=['AES', 'TEC', 'VDE', 'TES', 'GES'],
).mark_line().encode(
    x=alt.X('Year:T', title='Рік', axis=alt.Axis(format='%Y')),
    y=alt.Y('value:Q', title='Електроенергія', axis=alt.Axis(labelExpr="(datum.value > 0) ? datum.value/1000000 + ' млн МВт' : datum.value")),
    color=alt.Color('key:N', legend=alt.Legend(title='Джерела Електронергії')),
    tooltip=[alt.Tooltip('value:Q', title='Електроенергія')],
)

chart1Point = alt.Chart(dfByYear, title='Як змінювалась структура генерації електроенергії за роками?').transform_fold(
    fold=['AES', 'TEC', 'VDE', 'TES', 'GES'],
).mark_point(
    filled=True,
).encode(
    x=alt.X('Year:T', title='Рік', axis=alt.Axis(format='%Y')),
    y=alt.Y('value:Q', title='Електроенергія', axis=alt.Axis(labelExpr="(datum.value > 0) ? datum.value/1000000 + ' млн МВт' : datum.value")),
    color=alt.Color('key:N', legend=alt.Legend(title='Джерела Електронергії')),
    tooltip=[alt.Tooltip('value:Q', title='Електроенергія (МВт)')],
)

chart1 = alt.LayerChart(
    layer=[chart1Line, chart1Point],
).properties(
    padding=25,
).configure_title(
    dy=-10,
)

chart1

alt.LayerChart(...)

### 2. Як залежить споживання електроенергії від дня року та години доби?

Для цієї візуалізації я вирішив використати Heatmap з декількох причин. По-перше, дані які потрібно відобразити тривимірні: "День"/"Година"/"Споживання". Цей фактор одразу наштовхнув мене на варіант Heatmap оскільки цей спосіб візуалізації здатен ефективно та читабельно відобразити такі дані.

Як альтернативу я коротко розглянув Bar Chart але цей варіант потребуватиме об'єднання часу і днів що дасть як результат неймовірно довгу по осі x та шалено громіздку візуалізацію. Heatmap набагато компактніша та найефективніше організовує такий великий набір даних. Лінійний графік теж стражадає від раніше згаданих проблем Bar Chart-а.

In [51]:
chart2 = alt.Chart(
    df,
    title='Як залежить споживання електроенергії від дня року та години доби?',
).properties(
    padding=25,
).configure_title(
    dy=-10,
).mark_rect().encode(
    x=alt.X('Date:T', axis=alt.Axis(format='%d/%m/%Y', title='День Року')),
    y=alt.Y('Hour:Q', axis=alt.Axis(labelExpr="datum.value + ':00'", title='Година')),
    color=alt.Color('CONSUMPTION:Q', legend=alt.Legend(title='Споживання Електронергії (МВт)')),
    tooltip=[
        alt.Tooltip('Date:T', title='День Року'), 
        alt.Tooltip('Hour:Q', title='Година'), 
        alt.Tooltip('CONSUMPTION:Q', title='Споживання Електронергії (МВт)'),
    ],
)

chart2

alt.Chart(...)

### 3. Як змінюється генерація електроенергії з різних джерел впродовж доби?

Мій хід думок під час вибору візуалізації аналогічний тому, який я описав у питанні №1. 

Варто вказати, що задля зменшення об'єму даних, які варто відобразити та їх змістовного узагальнення, я згрупував дані по стовпцю Hour для кожної дати та обчислив середнє значення для кожної години усіх дат.

In [52]:
dfByHour = df.groupby('Hour').mean().reset_index()

dfByHour

,Hour,AES,TEC,VDE,TES,GES,GAES_GEN,CONSUMPTION,GAES_PUMP,UK_BLR_RUS,UK_EURO,UK_MLD
0,1,9533.777865,1428.560813,164.178334,4883.335549,413.657020,0.267110,15192.907704,-519.715291,-27.019554,-529.801330,-53.713336
1,10,9548.000782,1500.823230,529.519750,5897.132968,1088.394212,183.693000,18075.062573,-14.957372,-21.870943,-431.079390,-69.773954
2,11,9544.527571,1498.516230,623.170512,5946.095424,1071.591709,211.008213,18206.456003,-28.322644,-25.803285,-436.043410,-67.322253
3,12,9540.784513,1495.648807,675.165037,5949.276887,1003.193586,179.543997,18124.038326,-39.904185,-42.897536,-444.844349,-63.445444
4,13,9534.712163,1493.262417,676.166601,5949.325381,958.490418,152.303090,18083.541650,-47.095853,4.560813,-445.675792,-64.720767
5,14,9533.659366,1496.995698,634.258897,5999.127884,1007.424716,149.698475,18118.730153,-39.780211,-20.344544,-446.412202,-67.594838
6,15,9533.448181,1500.651154,561.769261,6038.778256,978.023856,145.707079,18062.907313,-30.281580,-24.904967,-446.606179,-63.531482
7,16,9532.448181,1505.319906,459.357450,6081.178334,1047.989050,166.456785,18149.113805,-17.472429,11.902229,-441.369965,-62.199844
8,17,9533.361361,1511.021118,357.425108,6143.126320,1165.674619,296.931169,18342.023465,-5.068831,-25.161126,-436.182245,-56.843567
9,18,9534.044192,1515.531482,261.122409,6203.737583,1241.757137,414.490418,18512.817364,-0.978099,-20.015252,-426.400469,-59.864685


In [104]:
chart3Line = alt.Chart(dfByHour, title='Як змінюється генерація електроенергії з різних джерел впродовж доби?').transform_fold(
    fold=['AES', 'TEC', 'VDE', 'TES', 'GES'],
).mark_line().encode(
    x=alt.X('Hour:Q', title='Година'),
    y=alt.Y('value:Q', title='Електроенергія', axis=alt.Axis(labelExpr="datum.value + '  МВт'")),
    color=alt.Color('key:N', legend=alt.Legend(title='Джерела Електронергії')),
    tooltip=[alt.Tooltip('value:Q', title='Електроенергія')],
)

chart3Point = alt.Chart(dfByHour, title='Як змінюється генерація електроенергії з різних джерел впродовж доби?').transform_fold(
    fold=['AES', 'TEC', 'VDE', 'TES', 'GES'],
).mark_point(
    filled=True,
).encode(
    x=alt.X('Hour:Q', title='Година', axis=alt.Axis(labelExpr="datum.value + ':00'")),
    y=alt.Y('value:Q', title='Електроенергія', axis=alt.Axis(labelExpr="datum.value + '  МВт'")),
    color=alt.Color('key:N', legend=alt.Legend(title='Джерела Електронергії')),
    tooltip=[alt.Tooltip('value:Q', title='Електроенергія (МВт)')],
)

chart3 = alt.LayerChart(
    layer=[chart3Line, chart3Point],
).properties(
    padding=25,
).configure_title(
    dy=-10,
)

chart3

alt.LayerChart(...)

### 4. Як змінюється споживання електроенергії впродовж доби у розрізі місяців року та пір року?

Тут ми знову зіткнулися із тривимірними даними: "Місяць(звідки можна логічно отримати пору року)/Година/Споживання електроенергії". Щоб дати відповідь на це питання я знову використав Heatmap, який є найпоширенішим і одним з найкращих способів візуалізації даних із трьома вимірами.

In [65]:
chart4 = alt.Chart(
    df,
    title='Як змінюється споживання електроенергії впродовж доби у розрізі місяців року та пір року?',
).properties(
    padding=25,
).configure_title(
    dy=-10,
).mark_rect().encode(
    x=alt.X('Month:T', axis=alt.Axis(format='%m', title='Місяць Року')),
    y=alt.Y('Hour:Q', axis=alt.Axis(labelExpr="datum.value + ':00'", title='Година')),
    color=alt.Color('CONSUMPTION:Q', legend=alt.Legend(title='Споживання Електронергії (МВт)')),
    tooltip=[
        alt.Tooltip('Month:T', title='Місяць Року'), 
        alt.Tooltip('Hour:Q', title='Година'), 
        alt.Tooltip('CONSUMPTION:Q', title='Споживання Електронергії (МВт)'),
    ],
)

chart4

alt.Chart(...)

### 5. Як змінюється споживання електроенергії впродовж тижня?

Через малу кількість позицій на осі x та великі максимальні значення по осі y я вирішив обрати Bar Chart. Помітні прямокутні стовпці та їх висоту буде легше та швидше порівняти, лінійний графік тут буде зайвим (хоч варіант прийнятний, але менш візуально виразний). Графік із точками буде занадто дрібним та нечітким, тож, як на мене, Bar Chart тут оптимальний варіант.

Дані по днях тижня - середні значення усіх відповідних днів тижня усіх років/місяців/тижнів.

In [70]:
dfByWeekDay = df.groupby('Weekday').mean().reset_index()

dfByWeekDay

,Weekday,AES,TEC,VDE,TES,GES,GAES_GEN,CONSUMPTION,GAES_PUMP,UK_BLR_RUS,UK_EURO,UK_MLD
0,Friday,9579.025228,1484.365753,313.225000,5796.676941,857.865411,171.530594,17306.823402,-216.445091,-30.490982,-462.867580,-56.589041
1,Monday,9505.673744,1481.564269,324.839155,5748.284932,856.009590,165.092808,17168.106963,-215.249201,-39.979566,-477.055479,-54.516096
2,Saturday,9536.147945,1474.379110,309.726142,5321.225228,795.537215,164.089612,16743.430137,-224.048402,-17.203448,-433.781025,-58.571184
3,Sunday,9473.300228,1463.029110,314.762557,4939.880137,738.401484,133.053881,16192.543721,-232.761872,-24.244292,-442.706393,-51.103082
4,Thursday,9570.079007,1485.572518,322.571152,5881.498975,871.914504,173.323543,17396.080487,-224.198565,-27.259335,-469.617372,-56.773793
5,Tuesday,9537.503539,1485.169863,320.857078,5862.809247,847.925457,166.989726,17310.623288,-220.743836,-26.907078,-478.278653,-56.530137
6,Wednesday,9567.187500,1485.850751,323.049750,5894.444444,860.471084,171.185792,17384.582992,-223.743966,-31.963912,-473.098019,-58.015141


In [103]:
chart5 = alt.Chart(
    dfByWeekDay,
    title='Як змінюється споживання електроенергії впродовж доби у розрізі місяців року та пір року?',
).properties(
    padding=25,
    width=500,
    height=400,
).configure_title(
    dy=-10,
).mark_bar(
).encode(
    x=alt.X('Weekday:N', sort=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'], axis=alt.Axis(labelAngle=0, title='День Тижня')),
    y=alt.Y('CONSUMPTION:Q', axis=alt.Axis(labelPadding=5, title='Споживання Електроенергії (МВт)')),
    tooltip=[
        alt.Tooltip('Weekday:N', title='День Тижня'),
        alt.Tooltip('CONSUMPTION:Q', title='Споживання Електроенергії (МВт)'),
    ]
)

chart5

alt.Chart(...)